# SEC Form 4 Data Collation
Refer to full readme.md for SEC data here: https://www.sec.gov/files/insider_transactions_readme.pdf 



### Overview of Merging Steps

1. **Combine Derivative & Non-Derivative Transactions**  
    - Merge `DERIV_TRANS` and `NONDERIV_TRANS` by their common columns.  
    - Rename their primary key columns (`DERIV_TRANS_SK` or `NONDERIV_TRANS_SK`) to `TRANS_SK`.

2. **Filter & Clean Transactions**  
    - **Include only transactions in YEARS_THRESHOLD (2005,2021)**.  
    - **Calculate transaction amount** as *(shares × price per share)*

3. **Merge with Form 4 Submission Data** 
    - Each Form 4 submission can include up to 30 transactions.  
    - Merging adds **filing date**, **period of report**, and the **issuer’s CIK** (i.e., the company whose shares are being transacted).

4. **Merge with Reporting Owner Data**  
   - Includes the **owner’s name** and **relationship** details.  
   - Note that there are multi-owner filings. Refer to `NUM_RPTOWNERCIK_;` to see how many reporting owners filed 1 submission. the `;` is used as a delimter to separate the reporting owner details in the related columns.

5. **Filter out codes** 'P', 'S', 'J', 'V','I', 'M', 'G'
    - we have saved an unfiltered version too

#### Importing libraries and folders

In [1]:
#Required libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

#Data folders
RAW_DATA_FOLDER = "../../data_untracked/raw/sec_submissions/FINAL_RAW_DATA"
FINAL_DATA_FOLDER = "../../data_untracked/processed"
YEARS_THRESHOLD = (2005, 2021) 

## Form Submission Main data
submission_data = pd.read_csv(f"{RAW_DATA_FOLDER}/SUBMISSION.csv") # pk : ACCESSION_NUMBER

## Transaction for submissions, ACCESSION_NUMBER and (NON)DERIV_TRANS_SK are the primary keys
# One form (i.e. ACCESSION_NUMBER) can have multiple transactions (i.e. *_SK), transactions can be across multiple years, max 30 each 
# Duplicate *_SK keys are for different transactions, and there are max 2 of each duplicate _SK keys
nonderiv_trans_data = pd.read_csv(f"{RAW_DATA_FOLDER}/NONDERIV_TRANS.csv")
deriv_trans_data = pd.read_csv(f"{RAW_DATA_FOLDER}/DERIV_TRANS.csv")

## Holding info for each submission (what they have - After each transaction..?)
#nonderiv_holding_data = pd.read_csv(f"{RAW_DATA_FOLDER}/NONDERIV_HOLDING.tsv",sep='\t')
#deriv_holding_data = pd.read_csv(f"{RAW_DATA_FOLDER}/DERIV_HOLDING.tsv",sep='\t')

## Name info
reporting_owner_data = pd.read_csv(f"{RAW_DATA_FOLDER}/REPORTINGOWNER.csv")
#names_data = pd.read_csv(f"{RAW_DATA_FOLDER}/OWNER_SIGNATURE.csv") 

## Additional info, to match with '*_FN' columns in all other datasets based on matching ACCESSION_NUMBER
#footnotes_data = pd.read_csv(f"{RAW_DATA_FOLDER}/FOOTNOTES.csv")

/var/folders/qs/szbbnwjs3xqfhs8mjs1pb1xr0000gn/T/ipykernel_88613/2068783219.py:17: DtypeWarning: Columns (7,13,23) have mixed types. Specify dtype option on import or set low_memory=False.
  nonderiv_trans_data = pd.read_csv(f"{RAW_DATA_FOLDER}/NONDERIV_TRANS.csv")
/var/folders/qs/szbbnwjs3xqfhs8mjs1pb1xr0000gn/T/ipykernel_88613/2068783219.py:18: DtypeWarning: Columns (9,15,19,23,33,37) have mixed types. Specify dtype option on import or set low_memory=False.
  deriv_trans_data = pd.read_csv(f"{RAW_DATA_FOLDER}/DERIV_TRANS.csv")


#### Simple EDA

In [2]:
#Overview of data
dataframes = {
    'submission_data': submission_data, # (4147030, 13)
    'nonderiv_trans_data': nonderiv_trans_data, # (6827027, 28)
    'deriv_trans_data': deriv_trans_data, # (2564274, 42)
    #'nonderiv_holding_data': nonderiv_holding_data,
   # 'deriv_holding_data': deriv_holding_data,
    'reporting_owner_data': reporting_owner_data, #  (4500775, 13)
    #'names_data': names_data,
    #'footnotes_data': footnotes_data
}

for name, df in dataframes.items():
    shape = df.shape
    print(f"{name}, {shape}")

    df.drop_duplicates(inplace=True)
    if df.shape[0] != shape[0]:
       print(f"Duplicate rows removed: {shape[0] - df.shape[0]}")
    else:
       print("No duplicate rows")
    print()

submission_data, (4147030, 13)
No duplicate rows

nonderiv_trans_data, (6827027, 28)
No duplicate rows

deriv_trans_data, (2564274, 42)
No duplicate rows

reporting_owner_data, (4500775, 13)
No duplicate rows



#### Subset of columns required from each csv

In [4]:
SELECTED_TRANSACTION_COLS = ['ACCESSION_NUMBER', 'SECURITY_TITLE', 'TRANS_DATE', 'DEEMED_EXECUTION_DATE', 'TRANS_CODE', 'EQUITY_SWAP_INVOLVED',
                             'TRANS_TIMELINESS', 'TRANS_SHARES', 'TRANS_PRICEPERSHARE', 'TRANS_ACQUIRED_DISP_CD',
                             'SHRS_OWND_FOLWNG_TRANS', 'DIRECT_INDIRECT_OWNERSHIP', 'NATURE_OF_OWNERSHIP']
# partial primary keys: 'NONDERIV_TRANS_SK', 'DERIV_TRANS_SK'

SUBMISSION_COLS = ['ACCESSION_NUMBER', 'FILING_DATE', 'PERIOD_OF_REPORT', 'ISSUERCIK', 'ISSUERNAME', 'ISSUERTRADINGSYMBOL']

REPORTING_OWNER_COLS = ['RPTOWNERCIK', 'RPTOWNERNAME','RPTOWNER_RELATIONSHIP', 'RPTOWNER_TITLE'] # there is also address data and filenumber

#### Merge Transaction datasets

In [5]:
#Changing dtypes
nonderiv_trans_data['TRANS_DATE'] = pd.to_datetime(nonderiv_trans_data['TRANS_DATE'], errors='coerce')
deriv_trans_data['TRANS_DATE'] = pd.to_datetime(deriv_trans_data['TRANS_DATE'], errors='coerce')

## Cannot directly concatenate the two dataframes as they have different columns some with identical names
df1 = nonderiv_trans_data[['NONDERIV_TRANS_SK'] + SELECTED_TRANSACTION_COLS].copy().rename(columns={'NONDERIV_TRANS_SK':'TRANS_SK'})
df2 = deriv_trans_data[['DERIV_TRANS_SK'] + SELECTED_TRANSACTION_COLS].copy().rename(columns={'DERIV_TRANS_SK':'TRANS_SK'})
all_transaction_data = pd.concat([df1,df2], axis=0, ignore_index=True).reset_index(drop=True)
print(all_transaction_data.shape) # (9391301, 14)

(9391301, 14)


In [6]:
# 1. Select only transactions from 2005 to 2021
all_transaction_data = all_transaction_data[(all_transaction_data['TRANS_DATE'].dt.year >= YEARS_THRESHOLD[0]) & (all_transaction_data['TRANS_DATE'].dt.year <= YEARS_THRESHOLD[1])]
print(all_transaction_data.shape) # (8176007, 14)

(8176007, 14)


In [ ]:
## Create transaction amount column
all_transaction_data['trans_amt'] = all_transaction_data['TRANS_SHARES'] * all_transaction_data['TRANS_PRICEPERSHARE']

trans_amt_0 = all_transaction_data[all_transaction_data['trans_amt'] == 0].shape[0]
print(f"There are {trans_amt_0} transactions with amount 0, {trans_amt_0/(all_transaction_data.shape[0])*100:.2f}% of all transactions")
print("Note that we do NOT filter this out as it may still be informative")
# There are 2486197 transactions with amount 0, 30.41% of all transactions
# Note that we do NOT filter this out as it may still be informative

There are 2486197 transactions with amount 0, 30.41% of all transactions
Note that we do NOT filter this out as it may still be informative


## Merge with submission data

In [8]:
# Change dtypes of filing date
submission_data['FILING_DATE'] = pd.to_datetime(submission_data['FILING_DATE'], errors='coerce')

# Merge on ACCESSION_NUMBER
all_transaction_data_2 = all_transaction_data.merge(submission_data[SUBMISSION_COLS], on='ACCESSION_NUMBER', how='left')
print(all_transaction_data_2.shape)

all_transaction_data_2.head(2)

(8176007, 20)


,TRANS_SK,ACCESSION_NUMBER,SECURITY_TITLE,TRANS_DATE,DEEMED_EXECUTION_DATE,TRANS_CODE,EQUITY_SWAP_INVOLVED,TRANS_TIMELINESS,TRANS_SHARES,TRANS_PRICEPERSHARE,TRANS_ACQUIRED_DISP_CD,SHRS_OWND_FOLWNG_TRANS,DIRECT_INDIRECT_OWNERSHIP,NATURE_OF_OWNERSHIP,trans_amt,FILING_DATE,PERIOD_OF_REPORT,ISSUERCIK,ISSUERNAME,ISSUERTRADINGSYMBOL
0,2360796,0001179110-14-011078,Common Stock,2014-06-26,NaN,S,0,NaN,1046.0,28.17,D,43944.0,D,NaN,29465.82,2014-06-30,2014-06-26,926326,"OMNICELL, Inc",OMCL
1,2360797,0001179110-14-011078,Common Stock,2014-06-26,NaN,S,0,NaN,637.0,28.17,D,43307.0,D,NaN,17944.29,2014-06-30,2014-06-26,926326,"OMNICELL, Inc",OMCL


## Merge with reporting owner data

In [9]:
# Ensure that ';' is NOT in these columns, such that a split by ';' will give the exact number of elements
assert reporting_owner_data['RPTOWNERCIK'].map(str).str.contains(';').sum() == 0
assert reporting_owner_data['RPTOWNERNAME'].str.contains(';').sum() == 0
assert reporting_owner_data['RPTOWNER_RELATIONSHIP'].str.contains(';').sum() == 0

# Ensure that '#' is NOT in this columns, Note that ';' is present in too many of these rows
reporting_owner_data['RPTOWNER_TITLE'] = reporting_owner_data['RPTOWNER_TITLE'].astype(str).str.replace('#', '', regex=False)
assert reporting_owner_data['RPTOWNER_TITLE'].str.contains('#').sum() == 0

In [11]:
# Join multiple reporting owners for the same ACCESSION_NUMBER

def join_with_nan(series):
    # catch NaNs and convert to 'NaN' string, to ensure they are not removed and the count is correct
    return ';'.join(series.fillna('NaN').astype(str))

def hashtag_join_with_nan(series):
    # catch NaNs and convert to 'NaN' string, to ensure they are not removed and the count is correct
    return '#'.join(series.fillna('NaN').astype(str))

# Group by ACCESSION_NUMBER and join other fields as semicolon-separated strings
grouped_reporting_owner = reporting_owner_data.groupby('ACCESSION_NUMBER', as_index=False).agg({
    'RPTOWNERCIK': [join_with_nan, 'count'],
    'RPTOWNERNAME': join_with_nan,
    'RPTOWNER_RELATIONSHIP': join_with_nan,
    'RPTOWNER_TITLE': hashtag_join_with_nan
})

assert reporting_owner_data['ACCESSION_NUMBER'].nunique() == grouped_reporting_owner.shape[0]

# Flatten MultiIndex columns
grouped_reporting_owner.columns = ['ACCESSION_NUMBER', 'RPTOWNERCIK_;', 'NUM_RPTOWNERCIK', 'RPTOWNERNAME_;', 'RPTOWNER_RELATIONSHIP_;','RPTOWNER_TITLE_#']
print(grouped_reporting_owner.shape)
grouped_reporting_owner.head(2)

(4147030, 6)


,ACCESSION_NUMBER,RPTOWNERCIK_;,NUM_RPTOWNERCIK,RPTOWNERNAME_;,RPTOWNER_RELATIONSHIP_;,RPTOWNER_TITLE_#
0,0000001750-06-000002,1027741,1,STORCH DAVID P,"Director,Officer","Chairman, Pres., CEO, Director"
1,0000001750-06-000004,1027741,1,STORCH DAVID P,"Director,Officer","Chairman, Pres., CEO, Director"


In [13]:
all_transaction_direct_final = all_transaction_data_2.merge(grouped_reporting_owner, on='ACCESSION_NUMBER', how='left') 
print(all_transaction_direct_final.shape) # (8176007, 25)
assert all_transaction_direct_final.shape[0] == all_transaction_data_2.shape[0]

(8176007, 25)


# Filter out codes

In [ ]:
all_transaction_direct_final_filter = all_transaction_direct_final[all_transaction_direct_final['TRANS_CODE'].isin(['P', 'S', 'J', 'V','I','G'])]
all_transaction_direct_final_filter.shape # without M code (3546490, 25) (old With M code (5121953, 25) )

(3546490, 25)

## Save files as csv

In [15]:
#all_transaction_direct_final.to_csv(f"{FINAL_DATA_FOLDER}/all_transaction_direct_final.csv", index=False)
all_transaction_direct_final_filter.to_csv(f"{FINAL_DATA_FOLDER}/all_transaction_final_filter_2.csv", index=False)